In [65]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2

### Read and Data analyze

In [171]:
from utils.config import tag2pos_table
from collections import Counter, defaultdict
from multiprocessing import Pool
import pickle, itertools
import kenlm

model = kenlm.Model('/home/nlplab/jjc/gec/lm/coca.prune.bin')

In [155]:
with open('recommend.pickle', 'rb') as handle:
    info = pickle.load(handle)
    counts, ngrams, sentences = info['counts'], info['ngrams'], info['sentences']

In [156]:
count_groups = defaultdict(list)
ngram_groups = defaultdict(lambda: defaultdict(list))

for pattern_group_name, pattern_group_items in itertools.groupby(counts, lambda el: el[0].split(' ')[0]):
    pattern_group_items = list(pattern_group_items)
    count_groups[pattern_group_name].extend(pattern_group_items)
    
    for key in pattern_group_items:
        for ngram_group_name, ngram_group_items in itertools.groupby(ngrams[key], lambda el: el.split(' ')[0]):
            ngram_groups[key][ngram_group_name].extend(list(ngram_group_items))


In [157]:
### NOT GOOD
from utils.EGP import EGP
Egp = EGP()
### NOT GOOD

In [197]:
def lm(last_sent, ngram):
    ngs = ngram.split(' ', maxsplit=1)
    if len(ngs) > 1: 
        sentence = last_sent + ' ' + ngram.split(' ', maxsplit=1)[1]
    else:            
        sentence = last_sent
    score = model.score(sentence, bos=True, eos=False)
    return (ngram, score)


def normalize_tag(tag):
    return tag2pos_table[tag] if tag in tag2pos_table else tag


def normalize_pattern(headword, pattern):
    tags = pattern.split(' ')
    
    norm_pattern = [headword]
    for tag in tags[1:]:
        norm_pattern.append(normalize_tag(tag))
    
    return ' '.join(norm_pattern)


In [ ]:
def suggest_backwards():
    pass

def suggest_forwards():
    pass

def suggest_collocations():
    pass

In [ ]:
def auto_suggest(headword, pos, last_sent):
    related_patterns = count_groups[headword] + count_groups[pos] # get headword matched POS and first word
    related_patterns = [related_pat for related_pat in related_patterns if headword in ngram_groups[related_pat]]

    ### 不需要用 count?
    # merge same number rule
    total = Counter()
    for pattern, no in related_patterns: 
        for ngram in ngram_groups[(pattern, no)][headword]:
            total[no] += ngrams[(pattern, no)][ngram]
    top_k_keys = dict(total.most_common(3))

    # get top k patterns
    target_patterns = filter(lambda key: key[1] in top_k_keys, related_patterns)

    # normalize patterns
    target_norm_patterns = defaultdict(list)
    for pattern, no in target_patterns:
        target_norm_patterns[normalize_pattern(headword, pattern)].append((pattern, no))
        
    # retrieve ngrams example
    total = 0
    target_ngrams = []
    for norm_pattern in target_norm_patterns: # (pattern, no)
        scores = [lm(last_sent, ng) for pattern, no in target_norm_patterns[norm_pattern] 
                  for ng in ngram_groups[(pattern, no)][headword]] # get ngram in given patterns
        scores = sorted(scores, key=lambda x: x[1], reverse=True)[:3]

        t_ngrams = [ng[0] for ng in scores]
        avg = 1 / abs(sum([s[1] for s in scores]) / len(scores))
        total += avg        

        target_ngrams.append({
            'pattern': norm_pattern, 'pos': normalize_tag(pos),
            'no': no, 'level': Egp.get_level(no), 
            # 'count': counts[(pattern, no)], 
            'lm': avg,
            'category': Egp.get_category(no), 'subcategory': Egp.get_subcategory(no),
            'ngrams': t_ngrams, 
            'sentence': sentences[t_ngrams[0]][0] })

    # get means and sorted
    for ng in target_ngrams: 
        ng['lm'] = ng['lm'] / total
        
    print(target_ngrams)
    return target_ngrams


In [198]:
headword, pos = 'look', 'VBP'

auto_suggest(headword, pos, 'I look')

[{'pattern': 'look adj', 'pos': 'v', 'no': 119, 'level': 'B1', 'lm': 0.06458592839670545, 'category': 'ADVERBS', 'subcategory': 'adverbs and adverb phrases: types and meanings', 'ngrams': ['look like', 'look good', 'look more'], 'sentence': "and you ' ll look a bit more like ."}, {'pattern': 'look carefully', 'pos': 'v', 'no': 119, 'level': 'B1', 'lm': 0.04482502910044849, 'category': 'ADVERBS', 'subcategory': 'adverbs and adverb phrases: types and meanings', 'ngrams': ['look carefully'], 'sentence': "But it might do well to look very carefully at the ' s work ."}, {'pattern': 'look closely', 'pos': 'v', 'no': 119, 'level': 'B1', 'lm': 0.04872764564714522, 'category': 'ADVERBS', 'subcategory': 'adverbs and adverb phrases: types and meanings', 'ngrams': ['look closely'], 'sentence': 'Any such attempt has to look closely at their chastened but ultimately unchastenable hero , at his hostility , at his stylistic authority and command of the books he belongs to .'}, {'pattern': 'look specif

[{'pattern': 'look adj',
  'pos': 'v',
  'no': 119,
  'level': 'B1',
  'lm': 0.06458592839670545,
  'category': 'ADVERBS',
  'subcategory': 'adverbs and adverb phrases: types and meanings',
  'ngrams': ['look like', 'look good', 'look more'],
  'sentence': "and you ' ll look a bit more like ."},
 {'pattern': 'look carefully',
  'pos': 'v',
  'no': 119,
  'level': 'B1',
  'lm': 0.04482502910044849,
  'category': 'ADVERBS',
  'subcategory': 'adverbs and adverb phrases: types and meanings',
  'ngrams': ['look carefully'],
  'sentence': "But it might do well to look very carefully at the ' s work ."},
 {'pattern': 'look closely',
  'pos': 'v',
  'no': 119,
  'level': 'B1',
  'lm': 0.04872764564714522,
  'category': 'ADVERBS',
  'subcategory': 'adverbs and adverb phrases: types and meanings',
  'ngrams': ['look closely'],
  'sentence': 'Any such attempt has to look closely at their chastened but ultimately unchastenable hero , at his hostility , at his stylistic authority and command of the

In [200]:
count_groups['simply']

[('simply', 109),
 ('simply VB ', 119),
 ('simply VB RP a NN -LRB- CC a VBN NN -RRB- TO VB a NN PRP VBP RB VBN IN DT NN that',
  123),
 ('simply a JJ NN IN WRB your', 123),
 ('simply not VB VBG her', 123),
 ('simply VBZ', 119),
 ('simply VBG', 119),
 ('simply VBN IN a JJ NN IN VBG his', 123),
 ('simply DT NN', 121),
 ('simply ,', 135),
 ('simply IN these', 123),
 ('simply VBN IN their', 123),
 ('simply JJ', 107),
 ('simply VBP', 119),
 ('simply VBD', 119),
 ('simply popular', 42),
 ('simply JJ NN', 121),
 ('simply IN EX VBZ RB that', 123),
 ("simply MD RB VB VBN WRB PRP '' NN VBN this", 123),
 ('simply VBN', 119),
 ('simply VBN IN this', 123),
 ('simply NN', 121),
 ('simply JJ IN their', 123),
 ('simply VBZ that', 123),
 ('simply VBZ VBG a little IN your', 123),
 ('simply RB', 124),
 ('simply IN IN their', 123),
 ('simply IN IN its', 123),
 ('simply IN IN DT NN IN these', 123),
 ('simply IN DT CD NNS VBP JJ NNS IN NN NN CC their', 123),
 ('simply .', 135),
 ('simply VBD RB CC VBD their

In [175]:
model.score('I am nice things', bos=True, eos=False)

-9.724052429199219